# PPP Loans: EDA

In [9]:
import pandas as pd
import numpy as np
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
import geocoder as geo

## Q0: Load Data

1. PPP Loan Level PPP_loans - SBA
2. Unemployment Rate by County by Month - FRED
3. County FIPS crosswalk
4. County lat/long

scrape_coords.py

```python

def scrape_coords(url):
    
    from bs4 import BeautifulSoup
    import requests
    
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    
    table = soup.find_all('table')[0]
    
    coords = pd.read_html(str(table))[0]
    
    results = coords.loc[:,['FIPS','Latitude','Longitude']]
    
    results['FIPS'] = '0'+results['FIPS'].astype(str)
    
    return results
    
def main():
    
    import argparse
    
    parser = argparse.ArgumentParser()
    
    parser.add_argument("-p", "--page_name",help="wiki page to scrape",required=True)
    parser.add_argument('-o','--output_name',help='output file name',required=True)
    
    args = parser.parse_args()
    
    results = scrape_coords(args.page_name)
    
    results.to_csv(f'{args.output_name}.csv',index=False)
    
    ```

In [68]:
web_page = "https://en.wikipedia.org/wiki/User:Michael_J/County_table"

output_name = "county_to_coords"

subprocess.call(['python3','./scrape_coords.py','-p',web_page,'-o',output_name])

0

## Q1: What is the distribution of PPP loans across the US

* Calculate at county level
* Use KDE to show spatial distribution

**STEPS**
1. Load PPP data into python
2. Remove any rows without a valid city or state

load_PPP.py

```python

def add_county(data):
    
    import pandas as pd
    
    counties = pd.read_csv("./ZIP-COUNTY-FIPS_2017-06.csv", dtype='object')
    
    data['BorrowerZip'] = data['BorrowerZip'].str[:5]
    
    data = data.merge(counties, left_on = ['BorrowerState','BorrowerZip'], right_on = ['STATE','ZIP'])
    
    data.drop(['ZIP','COUNTYNAME','STATE',
               'CLASSFP','Gender','Veteran',
               'NonProfit','Race','Ethnicity'],axis=1,inplace=True)
    
    data.rename(columns={'STCOUNTYFP': 'BorrowerFIPS'},inplace=True)
    
    return data

def add_coords(data):
    
    import pandas as pd

    coords = pd.read_csv('./county_to_coords.csv',dtype='object')
    
    data = data.merge(coords,left_on=['BorrowerFIPS'], right_on=['FIPS'])
    
    data.drop('FIPS',axis=1,inplace=True)
    
    return data


def load_PPP(file_path):

    import pandas as pd
        
    data = pd.read_csv(file_path, nrows=100)
    
    data = data.loc[~(data['BorrowerCity'].isin(['','N/A',np.nan])) & ~(data['BorrowerState'].isin(['','N/A',np.nan])),]
    
    data = add_county(data)
    
    data = add_coords(data)
    
    return data

def main():
    
    import argparse
    
    parser = argparse.ArgumentParser()
    
    parser.add_argument("-f", "--file_name",help="file to load",required=True)
    parser.add_argument('-o','--output_name',help='output file name',required=True)
    
    args = parser.parse_args()
    
    PPP_loans = load_PPP(args.file_name)
    
    PPP_loans.to_csv(f'{args.output_name}.csv', index=False)

```

In [3]:
file_name = './public_150k_plus_220930.csv'
PPP_name = 'PPP_with_FIPS'

#subprocess.call(['python3','./py_files/load_PPP.py','-f',file_name,'-o',PPP_name])

In [4]:
PPP_loans = pd.read_csv(f'./{PPP_name}.csv')

In [10]:
PPP_loans['full_address'] = PPP_loans['BorrowerAddress'] + ',' + PPP_loans['BorrowerCity'] + ',' + PPP_loans['BorrowerState']

In [17]:
def add_coords(data, sample = 1):
    
    from geopy.geocoders import Nominatim
    from geopy.extra.rate_limiter import RateLimiter
    
    shuffled = data.sample(frac=sample)
    
    geolocator = Nominatim(user_agent = 'UCLA stats 202 Project',timeout = 10)
    
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds = 1)
    
    shuffled['coords'] = shuffled['full_address'].apply(geocode)
    
    return shuffled

In [19]:
map_subset = add_coords(PPP_loans, sample = 1/100)

map_subset

## Q2: What Factors Should be Predictors

In [140]:
county_sums = PPP_loans.groupby('BorrowerFIPS').sum()[['CurrentApprovalAmount','ForgivenessAmount']].reset_index()
county_avgs = PPP_loans.groupby('BorrowerFIPS').mean()[['CurrentApprovalAmount','ForgivenessAmount']].reset_index()

county_data = county_sums.merge(county_avgs, on='BorrowerFIPS', suffixes=['_sum','_avg'])

county_data

/tmp/ipykernel_11246/2567810464.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  county_sums = PPP_loans.groupby('BorrowerFIPS').sum()[['CurrentApprovalAmount','ForgivenessAmount']].reset_index()
/tmp/ipykernel_11246/2567810464.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  county_avgs = PPP_loans.groupby('BorrowerFIPS').mean()[['CurrentApprovalAmount','ForgivenessAmount']].reset_index()


,BorrowerFIPS,CurrentApprovalAmount_sum,ForgivenessAmount_sum,CurrentApprovalAmount_avg,ForgivenessAmount_avg
0,1001,4.675590e+07,4.368620e+07,403068.080603,390055.385536
1,1003,2.881709e+08,2.807233e+08,419462.781456,418990.030194
2,1005,1.155519e+07,1.112355e+07,412685.347857,411983.401481
3,1007,3.393507e+07,3.419068e+07,477958.751549,481558.866620
4,1009,8.541598e+07,8.541106e+07,459225.678763,459199.227796
...,...,...,...,...,...
3167,72151,3.290510e+06,3.325827e+06,658102.000000,665165.434000
3168,72153,7.589880e+06,7.266665e+06,421660.000000,454166.557500
3169,78010,4.553892e+07,4.530799e+07,450880.369703,453079.866700
3170,78020,4.468858e+06,4.347389e+06,262874.002941,271711.840625
